# RO Flowsheet with Energy Recovery Device Build

This tutorial will walk you through creating a reverse osmosis (RO) flowsheet with an energy recovery device (ERD) unit model from top to bottom - building on the RO flowsheet constructed during class in RO_flowsheet_build.ipynb. 

<center><img src="graphics/RO_with_ERD.png" width="600" /></center>


By the end of this tutorial you should be able to follow the workflow below to construct a simple flowsheet.

<center><img src="graphics/watertap-flowsheet-workflow.png" width="600" /></center>

## Step 1: Import Modules

<center><img src="graphics/workflow-step1.png" width="600" /></center>

In [ ]:
# Imports from Pyomo
from pyomo.environ import (
    ConcreteModel,
    Var,
    Constraint,
    Objective,
    value,
    assert_optimal_termination,
    TransformationFactory,
    units as pyunits,
)
from pyomo.network import Arc

# Imports from IDAES
from idaes.core import FlowsheetBlock
from idaes.models.unit_models import Feed, Product
from idaes.core.util.model_statistics import degrees_of_freedom
from idaes.core.util.scaling import calculate_scaling_factors, set_scaling_factor
from idaes.core.util.initialization import propagate_state

# Imports from WaterTAP
from watertap.property_models.seawater_prop_pack import SeawaterParameterBlock
from watertap.unit_models.pressure_changer import Pump
from watertap.unit_models.pressure_changer import EnergyRecoveryDevice
from watertap.unit_models.reverse_osmosis_0D import (
    ReverseOsmosis0D,
    ConcentrationPolarizationType,
    MassTransferCoefficient,
    PressureChangeType,
)
from watertap.core.solvers import get_solver

## Step 2: Build Model

<center><img src="graphics/workflow-step2abc.png" width="600" /></center>

### Step 2a: Create Model Object

In [ ]:
# Create a model object using ConcreteModel

### Step 2b: Add Flowsheet to Model

In [ ]:
# Add the flowsheet to the model using FlowsheetBlock
# and set the dynamic configuration to False

### Step 2c: Add Property Packages to Flowsheet

In [ ]:
# Add the Seawater property package

### Step 2d: Add Unit Models to Flowsheet

<center><img src="graphics/workflow-step2d.png" width="600" /></center>

Add the unit models for Feed, Pump, RO, ERD, and Product. Keep in mind that all unit models have at least one configuration argument `property_package` that must be specified. For the RO model, set the configuration arguments such that it accounts for pressure change and the PressureChangeType, MassTransferCoefficient, and ConcentrationPolarizationType are all calculated.

<details>
  <summary>Click the arrow for a hint!</summary>
    
Unit models are defined with the following syntax: `m.fs.UserDefinedName = UnitModel(property_package=m.fs.PropertyPackage)`
</details>

In [ ]:
# Add feed

# Add pump

# Add 0D reverse osmosis unit

# Add Energy Recovery Device - check the imports if you're not sure of the unit model's name

# Add product stream

### Step 2e: Define Unit Model Connectivity

<center><img src="graphics/workflow-step2e.png" width="600" /></center>

Next, establish the connections betweeen unit models by creating an `Arc` between the `Ports` on each unit model. 

<details>
  <summary>Click the arrow for hint #1!</summary>
    
Arcs are defined with the following syntax: `m.fs.UserDefinedName = Arc(source=m.fs.UnitModelName.port, destination=m.fs.UnitModelName.port)`
</details>

<details>
  <summary>Click the arrow for hint #2!</summary>
    
In the schematic at the top of this tutorial there are 4 arrows (streams), so there should be four Arcs
</details>

In [1]:
# Define the connectivity using Arcs

### Step 2f: Expand Arcs

<center><img src="graphics/workflow-step2f.png" width="600" /></center>

After establishing these connection, the arcs are expanded to create the constraints between all the state variables.

In [ ]:
# Use the TransformationFactory to expand the arcs

### Step 2g: Add Variables, Constraints, and Objectives

<center><img src="graphics/workflow-step2g.png" width="600" /></center>

The last step before model specification is to add any additional variables, constraints, or objectives that are desired on the model. This is an optional step, but allows the user to create custom relationships on the fly.

For demonstration purposes, below we create a variable and constraint that defines the RO flux in LMH. Uncomment the variable and constraint below, then create an objective function that will maximize the RO flux.

<details>
  <summary>Click the arrow for a hint!</summary>
    
The syntax for creating an objective function is: `m.fs.UserDefinedName = Objective(expr=m.fs.VariableName, sense="maximize" or "minimize")`
</details>

In [ ]:
# m.fs.RO.flux_LMH = Var(
#     initialize=20.0,
#     bounds=(0, 45),
#     units=pyunits.liter / (pyunits.m**2 * pyunits.hr),
#     doc="Water flux through the membrane in LMH",
# )

# m.fs.RO.flux_LMH_constraint = Constraint(
#     expr=m.fs.RO.flux_LMH
#     == pyunits.convert(
#         m.fs.RO.mixed_permeate[0].flow_vol_phase["Liq"] / m.fs.RO.area,
#         to_units=pyunits.liter / (pyunits.m**2 * pyunits.hr),
#     )
# )

# Write an objective function that maximizes the RO flux

## Step 3: Specify Model

<center><img src="graphics/workflow-step3.png" width="600" /></center>

As before, we must provide parameter values for all the degrees of freedom on the model. Note that the `product` block is fully specified by its connected arcs, so we don't need to explicitly specify any of its degrees of freedom.

For each of the following unit models, specify the following conditions:

**Feed**
- Volumetric liquid flow rate: 1e-3 m3/s
- Mass concentration of liquid TDS: 35 kg/m3
- Pressure: 101325 Pa
- Temperature: 298.15 K

**Pump**
- Pump efficiency: 80%
- Outlet pressure: 75 bar

**Reverse Osmosis**
- Membrane water permeability: 4.12e-12
- Membrane salt permeability: 3.5e-8
- Fractional volumetric recovery of liquid: 0.5
- Feed side velocity: 0.15 m/s
- Feed side channel_height: 1e-3 m
- Feed side spacer_porosity: 0.97
- Permeate side pressure: 101325 Pa

**Energy Recovery Device**
- Pump efficiency: 95%
- Outlet pressure 101325 Pa

<details>
  <summary>Click the arrow for a hint!</summary>
    
**Feed**
- `flow_vol_phase["Liq"]`: 1e-3 m3/s
- `conc_mass_phase_comp["Liq", "TDS"]`: 35 kg/m3
- `pressure`: 101325 Pa
- `temperature`: 298.15 K

**Pump**
- `efficiency_pump`: 80%
- control volume outlet `pressure`: 75 bar

**Reverse Osmosis**
- `A_comp`: 4.12e-12
- `B_comp`: 3.5e-8
- `recovery_vol_phase["Liq"]`: 0.5
- feed side `velocity[0, 0]`: 0.15 m/s
- feed side `channel_height`1e-3 m
- feed side `spacer_porosity`: 0.97
- permeate `pressure`: 101325 Pa

**Energy Recovery Device**
- `efficiency_pump`: 95%
- outlet `pressure` 101325 Pa
</details>


In [ ]:
# Feed, 4 degrees of freedom

# Pump, 2 degrees of freedom

# RO unit, 7 degrees of freedom

# ERD, 2 degrees of freedom

# Print the degrees of freedom

## Step 4: Scale Model

<center><img src="graphics/workflow-step4.png" width="600" /></center>


Scaling ensures that all numerical quantities fall within similar, moderate magnitudes, improving model stability.

First we set default scaling factors for our state variables. Then we set custom scaling factors for the work done by the pump as well as the RO membrane area. Lastly, we use `calculate_scaling_factors` on the model, `m`, which will set scaling factors for the remaining variables on the model.

<details>
  <summary>Click the arrow for hint #1!</summary>
    
The syntax for setting default scaling factors on a property model is: `m.fs.PropertyModelName.set_default_scaling("VariableName", scaling factor, index=("Index for VariableName",...)`
</details>

<details>
  <summary>Click the arrow for hint #2!</summary>
    
The syntax for setting scaling factors on a unit model is: `set_scaling_factor(m.fs.UnitModelName.VariableName)` or `set_scaling_factor(m.fs.UnitModelName.control_volume.VariableName)`
</details>

In [ ]:
# Set default property values

# Set unit model values

# Calculate and propagate scaling factors

## Step 5: Initialize Model

<center><img src="graphics/workflow-step5.png" width="600" /></center>

After the model is scaled, we move to initialization. Initialization can be thought of solving each model in a vacuum, without any relation to the up or downstream effects. For the other steps in this workflow, the order with respect to the process flow did not matter. However, initialization must proceed from the beginning to the end of the treatment train.

Starting with the `feed` block, the model is set to an initial point. After solving the feed block, we _propagate the state_ from the feed `outlet` to the pump `inlet`. When we know the state of the pump inlet, we can solve the pump model and further propagate down stream, and so on.

<details>
  <summary>Click the arrow for hint #1!</summary>
    
The syntax for solving a model is: `solver.solve(m.fs.UnitModelName)`
</details>

<details>
  <summary>Click the arrow for hint #2!</summary>
    
The syntax for propagating the state is: `propagate_state(m.fs.ArcName)`
</details>

<details>
  <summary>Click the arrow for hint #3!</summary>
    
The syntax for initializing a unit model is: `m.fs.UnitModelName.initialize()`
</details>

In [ ]:
# Get WaterTAP solver
solver = get_solver()

# solve feed

# Propagate state from feed to pump

# Initialize pump

# Propagate state from pump to RO

# Initialize RO

# Propagate state from RO to ERD and product

# Initialize the product and ERD

## Step 6: Solve Model

<center><img src="graphics/workflow-step6.png" width="600" /></center>

In [ ]:
# Solve the model and assert the the model has reached optimal termination

## Step 7: Display the results

In [ ]:
# Report unit-level performance results for (1) the pump, (2) RO and (3) the energy recovery device, using the report() method for each unit on the flowsheet.

In [ ]:
# Unfix recovery and pump outlet pressure, re-solve, and assert optimal termination. Compare with reported results above.
# Recall: membrane flux will be maximized, yielding the corresponding pump discharge pressure and RO recovery

In [ ]:
# Repeat reporting of unit-level performance results for (1) the pump, (2) RO and (3) the energy recovery device, using the report() method for each unit on the flowsheet.